In [1]:
# ======================================================================
# CELL 1: Imports and Configuration
# ======================================================================
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, count, min as spark_min, max as spark_max, 
    when, isnan, isnull, trim, lower, upper, regexp_replace,
    to_date, date_format, lit, concat_ws, split, explode,
    avg, stddev, collect_list, struct, udf, lag, lead,
    sum as spark_sum, first, last, row_number, window,
    round as spark_round, abs as spark_abs, sqrt, log,
    countDistinct
)
from functools import reduce
from pyspark.sql.types import *
from pyspark.sql.window import Window
import os
from dotenv import load_dotenv
from google.cloud import storage, bigquery
from google.cloud.exceptions import NotFound

# Load environment variables
script_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()
project_root = os.path.dirname(script_dir) if '__file__' in globals() else os.path.dirname(os.getcwd())
env_path = os.path.join(project_root, ".env")
load_dotenv(dotenv_path=env_path)

GCS_BUCKET_NAME = os.getenv("GCS_BUCKET_NAME")
BIGQUERY_PROJECT = os.getenv("GCP_PROJECT_ID")
BIGQUERY_DATASET = os.getenv("BIGQUERY_DATASET")
BIGQUERY_TABLE_STOCKS = os.getenv("BIGQUERY_TABLE_STOCKS")

if not GCS_BUCKET_NAME:
    raise ValueError("GCS_BUCKET_NAME not found in .env file")
if not BIGQUERY_PROJECT:
    raise ValueError("GCP_PROJECT_ID not found in .env file")
if not BIGQUERY_DATASET:
    raise ValueError("BIGQUERY_DATASET not found in .env file")
if not BIGQUERY_TABLE_STOCKS:
    raise ValueError("BIGQUERY_TABLE_STOCKS not found in .env file")

# Resolve credentials path if it's relative
credentials_path = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
if credentials_path and not os.path.isabs(credentials_path):
    credentials_path = os.path.join(project_root, credentials_path)
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credentials_path

# Set JAVA_HOME to Java 17 for Spark
import subprocess
try:
    java_home = subprocess.check_output(['/usr/libexec/java_home', '-v', '17'], text=True).strip()
    os.environ['JAVA_HOME'] = java_home
    os.environ['PATH'] = f"{java_home}/bin:{os.environ.get('PATH', '')}"
    print(f"✅ JAVA_HOME set to: {java_home}")
except Exception as e:
    print(f"⚠️  Warning: Could not set JAVA_HOME automatically: {e}")
    print("   Make sure Java 17+ is installed and JAVA_HOME is set manually")

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("StockPriceTransformation") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

# Initialize BigQuery client
bq_client = bigquery.Client(project=BIGQUERY_PROJECT)

# Initialize GCS client
storage_client = storage.Client(project=BIGQUERY_PROJECT)

print("=" * 70)
print("STOCK PRICE TRANSFORMATION PIPELINE")
print("=" * 70)
print(f"GCS Bucket: {GCS_BUCKET_NAME}")
print(f"BigQuery Project: {BIGQUERY_PROJECT}")
print(f"BigQuery Dataset: {BIGQUERY_DATASET}")
print(f"BigQuery Table: {BIGQUERY_TABLE_STOCKS}")
print("=" * 70)


✅ JAVA_HOME set to: /Library/Java/JavaVirtualMachines/temurin-17.jdk/Contents/Home


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/14 07:06:56 WARN Utils: Your hostname, Evans-MacBook-Pro-2.local, resolves to a loopback address: 127.0.0.1; using 192.168.4.54 instead (on interface en0)
25/11/14 07:06:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/14 07:06:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


STOCK PRICE TRANSFORMATION PIPELINE
GCS Bucket: stock_sentiment_pipeline
BigQuery Project: solid-coral-469323-i0
BigQuery Dataset: stock_sentiment
BigQuery Table: stock_prices_silver


In [5]:
# ======================================================================
# CELL 2: Load Raw Stock Data from GCS Bronze Layer
# ======================================================================
from google.cloud import storage
import pandas as pd
import tempfile
import shutil

BRONZE_PRICES_PATH = "bronze/yfinance_prices"

print("=" * 70)
print("LOADING DATA FROM GCS")
print("=" * 70)
print(f"Bucket: {GCS_BUCKET_NAME}")
print(f"Path: {BRONZE_PRICES_PATH}")
print()

# Initialize GCS client
bucket = storage_client.bucket(GCS_BUCKET_NAME)

# Load per-ticker files (user confirmed they don't have a combined file)
print("Loading per-ticker files...")
# List all parquet files in ticker subdirectories
blobs = bucket.list_blobs(prefix=f"{BRONZE_PRICES_PATH}/")
parquet_files = [blob.name for blob in blobs if blob.name.endswith('.parquet') and '/prices_' in blob.name]

print(f"Found {len(parquet_files)} parquet files")

# Create a temporary directory to store downloaded files
temp_dir = tempfile.mkdtemp()
temp_files = []

try:
    # Download all files first
    print("Downloading files from GCS...")
    for file_path in parquet_files:
        try:
            blob = bucket.blob(file_path)
            temp_file_path = os.path.join(temp_dir, os.path.basename(file_path))
            blob.download_to_filename(temp_file_path)
            temp_files.append(temp_file_path)
        except Exception as e:
            print(f"⚠️  Error downloading {file_path}: {e}")
            continue
    
    if not temp_files:
        raise ValueError("No parquet files were successfully downloaded")
    
    print(f"Downloaded {len(temp_files)} files")
    print("Loading into Spark...")
    
    # Load all files into Spark at once (Spark can read multiple files)
    df = spark.read.parquet(*temp_files)
    
    # Force evaluation and cache to ensure data is loaded in memory before cleanup
    print("Caching DataFrame in memory...")
    df = df.cache()
    record_count = df.count()
    
    print(f"✅ Loaded {record_count:,} records")
    print()
    print("Schema:")
    df.printSchema()
    print()
    print("Sample data:")
    df.show(5, truncate=False)
    
finally:
    # Clean up temporary directory
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
        print(f"\n✅ Cleaned up temporary files")


LOADING DATA FROM GCS
Bucket: stock_sentiment_pipeline
Path: bronze/yfinance_prices

Loading per-ticker files...
Found 15 parquet files
Downloaded 15 files
Loading into Spark...
Caching DataFrame in memory...
✅ Loaded 3,750 records

Schema:
root
 |-- symbol: string (nullable = true)
 |-- date: string (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: long (nullable = true)


Sample data:
+------+----------+------------------+------------------+------------------+------------------+--------+
|symbol|date      |open              |high              |low               |close             |volume  |
+------+----------+------------------+------------------+------------------+------------------+--------+
|AMD   |2024-11-13|142.86000061035156|144.49000549316406|139.07000732421875|139.3000030517578 |35146600|
|AMD   |2024-11-14|140.33999633789062|141.39999389648438|138.5599

In [6]:
# ======================================================================
# CELL 3: Data Quality Check - Initial
# ======================================================================
print("=" * 70)
print("INITIAL DATA QUALITY CHECK")
print("=" * 70)

# Check for NULLs
print("\n1. NULL Value Counts:")
null_counts = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])
null_counts.show(vertical=True)

# Check data types
print("\n2. Data Types:")
for field in df.schema.fields:
    print(f"  {field.name}: {field.dataType}")

# Display sample rows
print("\n3. Sample Data (5 rows):")
df.show(5, truncate=False)

# Basic statistics
print("\n4. Record Count:")
print(f"  Total records: {df.count():,}")
print(f"  Unique symbols: {df.select('symbol').distinct().count()}")


INITIAL DATA QUALITY CHECK

1. NULL Value Counts:
-RECORD 0-----
 symbol | 0   
 date   | 0   
 open   | 0   
 high   | 0   
 low    | 0   
 close  | 0   
 volume | 0   


2. Data Types:
  symbol: StringType()
  date: StringType()
  open: DoubleType()
  high: DoubleType()
  low: DoubleType()
  close: DoubleType()
  volume: LongType()

3. Sample Data (5 rows):
+------+----------+------------------+------------------+------------------+------------------+--------+
|symbol|date      |open              |high              |low               |close             |volume  |
+------+----------+------------------+------------------+------------------+------------------+--------+
|AMD   |2024-11-13|142.86000061035156|144.49000549316406|139.07000732421875|139.3000030517578 |35146600|
|AMD   |2024-11-14|140.33999633789062|141.39999389648438|138.55999755859375|138.83999633789062|31681400|
|AMD   |2024-11-15|136.57000732421875|137.35000610351562|133.64999389648438|134.89999389648438|44217500|
|AMD   |

In [7]:
# ======================================================================
# CELL 4: Date Processing and Completeness Check
# ======================================================================
print("=" * 70)
print("DATE PROCESSING AND COMPLETENESS CHECK")
print("=" * 70)

# Convert date column to date type if it's a string
if df.schema["date"].dataType == StringType():
    print("Converting date column from string to date type...")
    df = df.withColumn("date", to_date(col("date"), "yyyy-MM-dd"))
else:
    print("Date column is already in date format")

# Get overall date range
date_range = df.agg(
    spark_min("date").alias("min_date"),
    spark_max("date").alias("max_date")
).collect()[0]

print(f"\nOverall Date Range:")
print(f"  Min date: {date_range['min_date']}")
print(f"  Max date: {date_range['max_date']}")

# Completeness check for each stock
print("\nCompleteness Check by Stock:")
completeness = df.groupBy("symbol").agg(
    count("date").alias("record_count"),
    spark_min("date").alias("min_date"),
    spark_max("date").alias("max_date"),
    countDistinct("date").alias("unique_dates")
).orderBy("symbol")

completeness.show(truncate=False)

# Filter out stocks with insufficient data (less than 200 trading days)
print("\nFiltering stocks with insufficient data (< 200 trading days)...")
df_filtered = df.join(
    completeness.filter(col("record_count") >= 200).select("symbol"),
    on="symbol",
    how="inner"
)

print(f"Records before filtering: {df.count():,}")
print(f"Records after filtering: {df_filtered.count():,}")
print(f"Stocks removed: {df.select('symbol').distinct().count() - df_filtered.select('symbol').distinct().count()}")

df = df_filtered


DATE PROCESSING AND COMPLETENESS CHECK
Converting date column from string to date type...

Overall Date Range:
  Min date: 2024-11-13
  Max date: 2025-11-12

Completeness Check by Stock:
+------+------------+----------+----------+------------+
|symbol|record_count|min_date  |max_date  |unique_dates|
+------+------------+----------+----------+------------+
|AAPL  |250         |2024-11-13|2025-11-12|250         |
|AMD   |250         |2024-11-13|2025-11-12|250         |
|AMZN  |250         |2024-11-13|2025-11-12|250         |
|GOOGL |250         |2024-11-13|2025-11-12|250         |
|INTC  |250         |2024-11-13|2025-11-12|250         |
|JPM   |250         |2024-11-13|2025-11-12|250         |
|META  |250         |2024-11-13|2025-11-12|250         |
|MSFT  |250         |2024-11-13|2025-11-12|250         |
|NVDA  |250         |2024-11-13|2025-11-12|250         |
|ORCL  |250         |2024-11-13|2025-11-12|250         |
|QQQ   |250         |2024-11-13|2025-11-12|250         |
|SPY   |250    

In [9]:
# ======================================================================
# CELL 5: Price Change Features
# ======================================================================
print("=" * 70)
print("CREATING PRICE CHANGE FEATURES")
print("=" * 70)

# Rename symbol to ticker for consistency
df = df.withColumnRenamed("symbol", "ticker")

# Define window partitioned by ticker, ordered by date (after rename)
window_spec = Window.partitionBy("ticker").orderBy("date")

# Create price change features
df = df.withColumn(
    "prev_close",
    lag("close", 1).over(window_spec)
).withColumn(
    "price_change",
    col("close") - col("prev_close")
).withColumn(
    "price_change_pct",
    when(col("prev_close") != 0, (col("price_change") / col("prev_close")) * 100)
    .otherwise(None)
)

print("✅ Created price change features:")
print("  - prev_close: Previous day's closing price")
print("  - price_change: Daily price change (close - prev_close)")
print("  - price_change_pct: Daily price change percentage")

df.select("ticker", "date", "close", "prev_close", "price_change", "price_change_pct").show(10, truncate=False)


CREATING PRICE CHANGE FEATURES
✅ Created price change features:
  - prev_close: Previous day's closing price
  - price_change: Daily price change (close - prev_close)
  - price_change_pct: Daily price change percentage
+------+----------+------------------+------------------+-------------------+--------------------+
|ticker|date      |close             |prev_close        |price_change       |price_change_pct    |
+------+----------+------------------+------------------+-------------------+--------------------+
|AAPL  |2024-11-13|225.1199951171875 |NULL              |NULL               |NULL                |
|AAPL  |2024-11-14|228.22000122070312|225.1199951171875 |3.100006103515625  |1.3770460957508013  |
|AAPL  |2024-11-15|225.0             |228.22000122070312|-3.220001220703125 |-1.4109198157391913 |
|AAPL  |2024-11-18|228.02000427246094|225.0             |3.0200042724609375 |1.34222412109375    |
|AAPL  |2024-11-19|228.27999877929688|228.02000427246094|0.2599945068359375 |0.114022674

In [10]:
# ======================================================================
# CELL 6: Moving Averages (SMA and EMA)
# ======================================================================
print("=" * 70)
print("CREATING MOVING AVERAGES")
print("=" * 70)

# Simple Moving Averages (SMA)
for period in [5, 10, 20, 50]:
    window_sma = Window.partitionBy("ticker").orderBy("date").rowsBetween(-(period-1), 0)
    df = df.withColumn(f"sma_{period}", avg("close").over(window_sma))

# Exponential Moving Averages (EMA) - using recursive calculation
# Note: EMA requires recursive calculation, so we'll use a UDF or iterative approach
# For PySpark, we'll calculate EMA using a window function approximation
# EMA = (Close - Previous EMA) * (2 / (Period + 1)) + Previous EMA

def calculate_ema(df, period):
    """Calculate EMA using window functions"""
    window_ema = Window.partitionBy("ticker").orderBy("date").rowsBetween(Window.unboundedPreceding, 0)
    
    # Initialize EMA with SMA for first period values
    window_sma = Window.partitionBy("ticker").orderBy("date").rowsBetween(-(period-1), 0)
    sma_col = avg("close").over(window_sma)
    
    # Calculate EMA using exponential smoothing factor
    alpha = 2.0 / (period + 1.0)
    
    # For simplicity, we'll use a recursive approach via collect_list and UDF
    # This is computationally expensive but accurate
    from pyspark.sql.functions import collect_list, array
    
    # Collect close prices in order
    df_with_list = df.withColumn(
        "close_list",
        collect_list("close").over(Window.partitionBy("ticker").orderBy("date").rowsBetween(Window.unboundedPreceding, 0))
    )
    
    # UDF to calculate EMA from list
    def calculate_ema_from_list(close_list, period):
        if len(close_list) == 0:
            return None
        alpha = 2.0 / (period + 1.0)
        ema = close_list[0]  # Start with first value
        for price in close_list[1:]:
            if price is not None:
                ema = (price * alpha) + (ema * (1 - alpha))
        return float(ema)
    
    ema_udf = udf(lambda x, p: calculate_ema_from_list(x, p) if x else None, FloatType())
    
    df_with_list = df_with_list.withColumn(f"ema_{period}", ema_udf(col("close_list"), lit(period)))
    df_with_list = df_with_list.drop("close_list")
    
    return df_with_list

# Calculate EMAs
for period in [5, 10, 20, 50]:
    df = calculate_ema(df, period)

print("✅ Created moving averages:")
print("  - SMA: 5, 10, 20, 50 day")
print("  - EMA: 5, 10, 20, 50 day")

df.select("ticker", "date", "close", "sma_5", "sma_20", "ema_5", "ema_20").show(10, truncate=False)


CREATING MOVING AVERAGES
✅ Created moving averages:
  - SMA: 5, 10, 20, 50 day
  - EMA: 5, 10, 20, 50 day


+------+----------+------------------+------------------+------------------+---------+---------+
|ticker|date      |close             |sma_5             |sma_20            |ema_5    |ema_20   |
+------+----------+------------------+------------------+------------------+---------+---------+
|AAPL  |2024-11-13|225.1199951171875 |225.1199951171875 |225.1199951171875 |225.12   |225.12   |
|AAPL  |2024-11-14|228.22000122070312|226.6699981689453 |226.6699981689453 |226.15334|225.41524|
|AAPL  |2024-11-15|225.0             |226.11333211263022|226.11333211263022|225.76889|225.37569|
|AAPL  |2024-11-18|228.02000427246094|226.5900001525879 |226.5900001525879 |226.51926|225.62753|
|AAPL  |2024-11-19|228.27999877929688|226.9279998779297 |226.9279998779297 |227.10617|225.88014|
|AAPL  |2024-11-20|229.0             |227.7040008544922 |227.27333323160806|227.73744|226.17728|
|AAPL  |2024-11-21|228.52000427246094|227.76400146484374|227.45142909458704|227.9983 |226.40039|
|AAPL  |2024-11-22|229.8699951

In [11]:
# ======================================================================
# CELL 7: Volatility Measures
# ======================================================================
print("=" * 70)
print("CREATING VOLATILITY MEASURES")
print("=" * 70)

window_spec = Window.partitionBy("ticker").orderBy("date")

# Daily volatility (high - low)
df = df.withColumn("daily_range", col("high") - col("low"))
df = df.withColumn("daily_range_pct", (col("daily_range") / col("close")) * 100)

# Rolling volatility (standard deviation of returns)
for period in [5, 10, 20]:
    window_vol = Window.partitionBy("ticker").orderBy("date").rowsBetween(-(period-1), 0)
    df = df.withColumn(
        f"volatility_{period}d",
        stddev("price_change_pct").over(window_vol)
    )

print("✅ Created volatility measures:")
print("  - daily_range: High - Low")
print("  - daily_range_pct: Daily range as percentage of close")
print("  - volatility_5d, volatility_10d, volatility_20d: Rolling standard deviation of returns")

df.select("ticker", "date", "close", "daily_range", "daily_range_pct", "volatility_10d").show(10, truncate=False)


CREATING VOLATILITY MEASURES
✅ Created volatility measures:
  - daily_range: High - Low
  - daily_range_pct: Daily range as percentage of close
  - volatility_5d, volatility_10d, volatility_20d: Rolling standard deviation of returns
+------+----------+------------------+------------------+------------------+------------------+
|ticker|date      |close             |daily_range       |daily_range_pct   |volatility_10d    |
+------+----------+------------------+------------------+------------------+------------------+
|AAPL  |2024-11-13|225.1199951171875 |3.8899993896484375|1.7279670726820493|NULL              |
|AAPL  |2024-11-14|228.22000122070312|3.8699951171875   |1.6957300396493167|NULL              |
|AAPL  |2024-11-15|225.0             |2.649993896484375 |1.1777750651041667|1.9713896017315078|
|AAPL  |2024-11-18|228.02000427246094|4.57000732421875  |2.0042133315452673|1.5996753853240155|
|AAPL  |2024-11-19|228.27999877929688|3.5               |1.5332048443647623|1.3160206617366768|

In [12]:
# ======================================================================
# CELL 8: Volume Analysis and VWAP
# ======================================================================
print("=" * 70)
print("CREATING VOLUME ANALYSIS AND VWAP")
print("=" * 70)

window_spec = Window.partitionBy("ticker").orderBy("date")

# Volume moving averages
for period in [5, 10, 20]:
    window_vol = Window.partitionBy("ticker").orderBy("date").rowsBetween(-(period-1), 0)
    df = df.withColumn(f"volume_sma_{period}", avg("volume").over(window_vol))

# Volume ratio (current volume / average volume)
df = df.withColumn("volume_ratio", col("volume") / col("volume_sma_20"))

# VWAP (Volume Weighted Average Price) - cumulative
# VWAP = Sum(Price * Volume) / Sum(Volume) over the day
# For daily data, we'll calculate rolling VWAP
window_vwap = Window.partitionBy("ticker").orderBy("date").rowsBetween(Window.unboundedPreceding, 0)
df = df.withColumn(
    "vwap_cumulative",
    spark_sum(col("close") * col("volume")).over(window_vwap) / spark_sum(col("volume")).over(window_vwap)
)

# Rolling VWAP (20-day)
window_vwap_20 = Window.partitionBy("ticker").orderBy("date").rowsBetween(-19, 0)
df = df.withColumn(
    "vwap_20d",
    spark_sum(col("close") * col("volume")).over(window_vwap_20) / spark_sum(col("volume")).over(window_vwap_20)
)

print("✅ Created volume analysis and VWAP:")
print("  - volume_sma_5, volume_sma_10, volume_sma_20: Volume moving averages")
print("  - volume_ratio: Current volume / 20-day average volume")
print("  - vwap_cumulative: Cumulative VWAP")
print("  - vwap_20d: 20-day rolling VWAP")

df.select("ticker", "date", "close", "volume", "volume_ratio", "vwap_20d").show(10, truncate=False)


CREATING VOLUME ANALYSIS AND VWAP
✅ Created volume analysis and VWAP:
  - volume_sma_5, volume_sma_10, volume_sma_20: Volume moving averages
  - volume_ratio: Current volume / 20-day average volume
  - vwap_cumulative: Cumulative VWAP
  - vwap_20d: 20-day rolling VWAP
+------+----------+------------------+--------+------------------+------------------+
|ticker|date      |close             |volume  |volume_ratio      |vwap_20d          |
+------+----------+------------------+--------+------------------+------------------+
|AAPL  |2024-11-13|225.1199951171875 |48566200|1.0               |225.1199951171875 |
|AAPL  |2024-11-14|228.22000122070312|44923900|0.9610407946937697|226.6096112818266 |
|AAPL  |2024-11-15|225.0             |47923700|1.0166695188164097|226.06413037270121|
|AAPL  |2024-11-18|228.02000427246094|44633700|0.9596194520216611|226.53335403267894|
|AAPL  |2024-11-19|228.27999877929688|36211800|0.8146295790547348|226.81792772761804|
|AAPL  |2024-11-20|229.0             |35169

In [13]:
# ======================================================================
# CELL 9: RSI (Relative Strength Index)
# ======================================================================
print("=" * 70)
print("CREATING RSI (14-DAY)")
print("=" * 70)

# RSI = 100 - (100 / (1 + RS))
# RS = Average Gain / Average Loss over 14 periods
# We'll use Wilder's smoothing method

window_spec = Window.partitionBy("ticker").orderBy("date")

# Calculate price changes
df = df.withColumn("price_change_for_rsi", col("close") - lag("close", 1).over(window_spec))

# Separate gains and losses
df = df.withColumn("gain", when(col("price_change_for_rsi") > 0, col("price_change_for_rsi")).otherwise(0))
df = df.withColumn("loss", when(col("price_change_for_rsi") < 0, -col("price_change_for_rsi")).otherwise(0))

# Calculate average gain and loss using Wilder's smoothing (EMA-like)
# First, calculate initial average (SMA of first 14 periods)
window_rsi_init = Window.partitionBy("ticker").orderBy("date").rowsBetween(-13, 0)
df = df.withColumn("avg_gain_init", avg("gain").over(window_rsi_init))
df = df.withColumn("avg_loss_init", avg("loss").over(window_rsi_init))

# Then apply Wilder's smoothing: New Avg = (Previous Avg * 13 + Current Value) / 14
# This is equivalent to EMA with alpha = 1/14
def calculate_wilders_avg(df, col_name, init_col):
    """Calculate Wilder's smoothed average"""
    from pyspark.sql.functions import collect_list
    
    window_all = Window.partitionBy("ticker").orderBy("date").rowsBetween(Window.unboundedPreceding, 0)
    df = df.withColumn(
        f"{col_name}_list",
        collect_list(col(col_name)).over(window_all)
    )
    df = df.withColumn(
        f"{init_col}_list",
        collect_list(col(init_col)).over(window_all)
    )
    
    def wilders_smoothing(value_list, init_list, period=14):
        if len(value_list) == 0:
            return None
        if len(value_list) <= period:
            return init_list[-1] if len(init_list) > 0 else None
        
        # Start with initial average
        avg = init_list[period-1] if len(init_list) >= period else value_list[0] if value_list[0] else 0
        
        # Apply Wilder's smoothing for remaining values
        for i in range(period, len(value_list)):
            if value_list[i] is not None:
                avg = (avg * (period - 1) + value_list[i]) / period
        return float(avg)
    
    wilders_udf = udf(lambda v, i: wilders_smoothing(v, i) if v and i else None, FloatType())
    df = df.withColumn(f"avg_{col_name}_wilders", wilders_udf(col(f"{col_name}_list"), col(f"{init_col}_list")))
    df = df.drop(f"{col_name}_list", f"{init_col}_list")
    return df

df = calculate_wilders_avg(df, "gain", "avg_gain_init")
df = calculate_wilders_avg(df, "loss", "avg_loss_init")

# Calculate RS and RSI
df = df.withColumn(
    "rs",
    when(col("avg_loss_wilders") != 0, col("avg_gain_wilders") / col("avg_loss_wilders")).otherwise(None)
)
df = df.withColumn("rsi_14", 100 - (100 / (1 + col("rs"))))

# Clean up intermediate columns
df = df.drop("price_change_for_rsi", "gain", "loss", "avg_gain_init", "avg_loss_init", "avg_gain_wilders", "avg_loss_wilders", "rs")

print("✅ Created RSI (14-day):")
print("  - rsi_14: Relative Strength Index (0-100)")

df.select("ticker", "date", "close", "rsi_14").show(10, truncate=False)


CREATING RSI (14-DAY)
✅ Created RSI (14-day):
  - rsi_14: Relative Strength Index (0-100)
+------+----------+------------------+-----------------+
|ticker|date      |close             |rsi_14           |
+------+----------+------------------+-----------------+
|AAPL  |2024-11-13|225.1199951171875 |NULL             |
|AAPL  |2024-11-14|228.22000122070312|NULL             |
|AAPL  |2024-11-15|225.0             |49.05067168039241|
|AAPL  |2024-11-18|228.02000427246094|65.52465500312854|
|AAPL  |2024-11-19|228.27999877929688|66.4583423555151 |
|AAPL  |2024-11-20|229.0             |68.79845992796437|
|AAPL  |2024-11-21|228.52000427246094|65.74077908720065|
|AAPL  |2024-11-22|229.8699951171875 |69.54733492242502|
|AAPL  |2024-11-25|232.8699951171875 |75.57756923147087|
|AAPL  |2024-11-26|235.05999755859375|78.66206590219397|
+------+----------+------------------+-----------------+
only showing top 10 rows


In [14]:
# ======================================================================
# CELL 10: Momentum Features (5-day and 20-day)
# ======================================================================
print("=" * 70)
print("CREATING MOMENTUM FEATURES")
print("=" * 70)

window_spec = Window.partitionBy("ticker").orderBy("date")

# 5-day momentum (price change over 5 days)
df = df.withColumn("momentum_5d", col("close") - lag("close", 5).over(window_spec))
df = df.withColumn("momentum_5d_pct", ((col("close") - lag("close", 5).over(window_spec)) / lag("close", 5).over(window_spec)) * 100)

# 20-day momentum (price change over 20 days)
df = df.withColumn("momentum_20d", col("close") - lag("close", 20).over(window_spec))
df = df.withColumn("momentum_20d_pct", ((col("close") - lag("close", 20).over(window_spec)) / lag("close", 20).over(window_spec)) * 100)

print("✅ Created momentum features:")
print("  - momentum_5d: 5-day price change")
print("  - momentum_5d_pct: 5-day price change percentage")
print("  - momentum_20d: 20-day price change")
print("  - momentum_20d_pct: 20-day price change percentage")

df.select("ticker", "date", "close", "momentum_5d", "momentum_5d_pct", "momentum_20d", "momentum_20d_pct").show(10, truncate=False)


CREATING MOMENTUM FEATURES
✅ Created momentum features:
  - momentum_5d: 5-day price change
  - momentum_5d_pct: 5-day price change percentage
  - momentum_20d: 20-day price change
  - momentum_20d_pct: 20-day price change percentage
+------+----------+------------------+------------------+-------------------+------------+----------------+
|ticker|date      |close             |momentum_5d       |momentum_5d_pct    |momentum_20d|momentum_20d_pct|
+------+----------+------------------+------------------+-------------------+------------+----------------+
|AAPL  |2024-11-13|225.1199951171875 |NULL              |NULL               |NULL        |NULL            |
|AAPL  |2024-11-14|228.22000122070312|NULL              |NULL               |NULL        |NULL            |
|AAPL  |2024-11-15|225.0             |NULL              |NULL               |NULL        |NULL            |
|AAPL  |2024-11-18|228.02000427246094|NULL              |NULL               |NULL        |NULL            |
|AAPL  |20

In [15]:
# ======================================================================
# CELL 11: Bollinger Bands
# ======================================================================
print("=" * 70)
print("CREATING BOLLINGER BANDS")
print("=" * 70)

# Bollinger Bands: Middle band (SMA), Upper band (SMA + 2*std), Lower band (SMA - 2*std)
# Typically 20-day period with 2 standard deviations

period = 20
num_std = 2

window_bb = Window.partitionBy("ticker").orderBy("date").rowsBetween(-(period-1), 0)

# Middle band (20-day SMA)
df = df.withColumn("bb_middle", avg("close").over(window_bb))

# Standard deviation
df = df.withColumn("bb_std", stddev("close").over(window_bb))

# Upper and lower bands
df = df.withColumn("bb_upper", col("bb_middle") + (lit(num_std) * col("bb_std")))
df = df.withColumn("bb_lower", col("bb_middle") - (lit(num_std) * col("bb_std")))

# Bollinger Band width and %B
df = df.withColumn("bb_width", col("bb_upper") - col("bb_lower"))
df = df.withColumn(
    "bb_percent_b",
    when(col("bb_upper") != col("bb_lower"), (col("close") - col("bb_lower")) / (col("bb_upper") - col("bb_lower")))
    .otherwise(None)
)

print("✅ Created Bollinger Bands:")
print("  - bb_middle: 20-day SMA (middle band)")
print("  - bb_upper: Upper band (SMA + 2*std)")
print("  - bb_lower: Lower band (SMA - 2*std)")
print("  - bb_width: Band width")
print("  - bb_percent_b: %B indicator (0-1)")

df.select("ticker", "date", "close", "bb_middle", "bb_upper", "bb_lower", "bb_percent_b").show(10, truncate=False)


CREATING BOLLINGER BANDS
✅ Created Bollinger Bands:
  - bb_middle: 20-day SMA (middle band)
  - bb_upper: Upper band (SMA + 2*std)
  - bb_lower: Lower band (SMA - 2*std)
  - bb_width: Band width
  - bb_percent_b: %B indicator (0-1)
+------+----------+------------------+------------------+------------------+------------------+-------------------+
|ticker|date      |close             |bb_middle         |bb_upper          |bb_lower          |bb_percent_b       |
+------+----------+------------------+------------------+------------------+------------------+-------------------+
|AAPL  |2024-11-13|225.1199951171875 |225.1199951171875 |NULL              |NULL              |NULL               |
|AAPL  |2024-11-14|228.22000122070312|226.6699981689453 |231.0540688439765 |222.28592749391413|0.6767766952966364 |
|AAPL  |2024-11-15|225.0             |226.11333211263022|229.76416256896135|222.46250165629908|0.34752344350865183|
|AAPL  |2024-11-18|228.02000427246094|226.5900001525879 |230.12851783738

In [16]:
# ======================================================================
# CELL 12: MACD (Moving Average Convergence Divergence)
# ======================================================================
print("=" * 70)
print("CREATING MACD")
print("=" * 70)

# MACD = 12-day EMA - 26-day EMA
# Signal line = 9-day EMA of MACD
# Histogram = MACD - Signal

window_spec = Window.partitionBy("ticker").orderBy("date")

# Calculate 12-day and 26-day EMA (reusing EMA function)
df = calculate_ema(df, 12)
df = calculate_ema(df, 26)
df = calculate_ema(df, 9)

# MACD line
df = df.withColumn("macd", col("ema_12") - col("ema_26"))

# Signal line (9-day EMA of MACD) - need to calculate EMA of MACD
# For simplicity, we'll use a 9-day SMA of MACD as signal line approximation
window_signal = Window.partitionBy("ticker").orderBy("date").rowsBetween(-8, 0)
df = df.withColumn("macd_signal", avg("macd").over(window_signal))

# Histogram
df = df.withColumn("macd_histogram", col("macd") - col("macd_signal"))

print("✅ Created MACD:")
print("  - macd: MACD line (12-day EMA - 26-day EMA)")
print("  - macd_signal: Signal line (9-day EMA of MACD)")
print("  - macd_histogram: MACD - Signal")

df.select("ticker", "date", "close", "macd", "macd_signal", "macd_histogram").show(10, truncate=False)


CREATING MACD
✅ Created MACD:
  - macd: MACD line (12-day EMA - 26-day EMA)
  - macd_signal: Signal line (9-day EMA of MACD)
  - macd_histogram: MACD - Signal
+------+----------+------------------+----------+-------------------+-------------------+
|ticker|date      |close             |macd      |macd_signal        |macd_histogram     |
+------+----------+------------------+----------+-------------------+-------------------+
|AAPL  |2024-11-13|225.1199951171875 |0.0       |0.0                |0.0                |
|AAPL  |2024-11-14|228.22000122070312|0.2472992 |0.12364959716796875|0.12364959716796875|
|AAPL  |2024-11-15|225.0             |0.18135071|0.14288330078125   |0.0384674072265625 |
|AAPL  |2024-11-18|228.02000427246094|0.36854553|0.19929885864257812|0.16924667358398438|
|AAPL  |2024-11-19|228.27999877929688|0.5317383 |0.2657867431640625 |0.2659515380859375 |
|AAPL  |2024-11-20|229.0             |0.7109833 |0.3399861653645833 |0.3709971110026042 |
|AAPL  |2024-11-21|228.52000427

In [17]:
# ======================================================================
# CELL 13: Risk Metrics and Returns Calculations
# ======================================================================
print("=" * 70)
print("CREATING RISK METRICS AND RETURNS")
print("=" * 70)

window_spec = Window.partitionBy("ticker").orderBy("date")

# Daily returns (already have price_change_pct, but let's add log returns)
df = df.withColumn(
    "log_return",
    when(col("prev_close") > 0, log(col("close") / col("prev_close")))
    .otherwise(None)
)

# Cumulative returns
window_cumulative = Window.partitionBy("ticker").orderBy("date").rowsBetween(Window.unboundedPreceding, 0)
df = df.withColumn(
    "cumulative_return",
    spark_sum("log_return").over(window_cumulative)
)

# Rolling Sharpe ratio (assuming risk-free rate = 0 for simplicity)
# Sharpe = (Mean Return / Std Dev of Returns) * sqrt(252) for annualized
for period in [20, 60]:
    window_sharpe = Window.partitionBy("ticker").orderBy("date").rowsBetween(-(period-1), 0)
    mean_return = avg("log_return").over(window_sharpe)
    std_return = stddev("log_return").over(window_sharpe)
    df = df.withColumn(
        f"sharpe_ratio_{period}d",
        when(std_return != 0, (mean_return / std_return) * sqrt(lit(252)))
        .otherwise(None)
    )

# Maximum drawdown (rolling)
window_dd = Window.partitionBy("ticker").orderBy("date").rowsBetween(Window.unboundedPreceding, 0)
df = df.withColumn("running_max", spark_max("close").over(window_dd))
df = df.withColumn("drawdown", (col("close") - col("running_max")) / col("running_max"))
df = df.withColumn("drawdown_pct", col("drawdown") * 100)

print("✅ Created risk metrics and returns:")
print("  - log_return: Logarithmic daily return")
print("  - cumulative_return: Cumulative log return")
print("  - sharpe_ratio_20d, sharpe_ratio_60d: Rolling Sharpe ratios")
print("  - drawdown: Current drawdown from peak")
print("  - drawdown_pct: Drawdown percentage")

df.select("ticker", "date", "close", "log_return", "cumulative_return", "drawdown_pct").show(10, truncate=False)


CREATING RISK METRICS AND RETURNS
✅ Created risk metrics and returns:
  - log_return: Logarithmic daily return
  - cumulative_return: Cumulative log return
  - sharpe_ratio_20d, sharpe_ratio_60d: Rolling Sharpe ratios
  - drawdown: Current drawdown from peak
  - drawdown_pct: Drawdown percentage
+------+----------+------------------+---------------------+---------------------+--------------------+
|ticker|date      |close             |log_return           |cumulative_return    |drawdown_pct        |
+------+----------+------------------+---------------------+---------------------+--------------------+
|AAPL  |2024-11-13|225.1199951171875 |NULL                 |NULL                 |0.0                 |
|AAPL  |2024-11-14|228.22000122070312|0.013676509679075834 |0.013676509679075834 |0.0                 |
|AAPL  |2024-11-15|225.0             |-0.014209679150913371|-5.331694718375378E-4|-1.4109198157391913 |
|AAPL  |2024-11-18|228.02000427246094|0.013332960938373547 |0.01279979146653600

In [18]:
# ======================================================================
# CELL 14: Price Patterns
# ======================================================================
print("=" * 70)
print("CREATING PRICE PATTERNS")
print("=" * 70)

window_spec = Window.partitionBy("ticker").orderBy("date")

# Doji pattern (open and close are very close)
df = df.withColumn(
    "is_doji",
    when(spark_abs(col("open") - col("close")) / col("close") < 0.001, 1).otherwise(0)
)

# Hammer pattern (long lower wick, small body)
body = spark_abs(col("close") - col("open"))
lower_wick = when(col("open") > col("close"), col("open") - col("low")).otherwise(col("close") - col("low"))
upper_wick = when(col("open") > col("close"), col("high") - col("open")).otherwise(col("high") - col("close"))
df = df.withColumn(
    "is_hammer",
    when((lower_wick > 2 * body) & (upper_wick < body * 0.5), 1).otherwise(0)
)

# Engulfing pattern (current candle engulfs previous)
prev_open = lag("open", 1).over(window_spec)
prev_close = lag("close", 1).over(window_spec)
df = df.withColumn(
    "is_bullish_engulfing",
    when(
        (prev_close < prev_open) &  # Previous was bearish
        (col("close") > col("open")) &  # Current is bullish
        (col("open") < prev_close) &  # Current opens below previous close
        (col("close") > prev_open),  # Current closes above previous open
        1
    ).otherwise(0)
)
df = df.withColumn(
    "is_bearish_engulfing",
    when(
        (prev_close > prev_open) &  # Previous was bullish
        (col("close") < col("open")) &  # Current is bearish
        (col("open") > prev_close) &  # Current opens above previous close
        (col("close") < prev_open),  # Current closes below previous open
        1
    ).otherwise(0)
)

# Price position relative to daily range
df = df.withColumn(
    "price_position",
    when(col("high") != col("low"), (col("close") - col("low")) / (col("high") - col("low")))
    .otherwise(0.5)
)

print("✅ Created price patterns:")
print("  - is_doji: Doji candlestick pattern")
print("  - is_hammer: Hammer candlestick pattern")
print("  - is_bullish_engulfing: Bullish engulfing pattern")
print("  - is_bearish_engulfing: Bearish engulfing pattern")
print("  - price_position: Close price position in daily range (0-1)")

df.select("ticker", "date", "open", "high", "low", "close", "is_doji", "is_hammer", "price_position").show(10, truncate=False)


CREATING PRICE PATTERNS
✅ Created price patterns:
  - is_doji: Doji candlestick pattern
  - is_hammer: Hammer candlestick pattern
  - is_bullish_engulfing: Bullish engulfing pattern
  - is_bearish_engulfing: Bearish engulfing pattern
  - price_position: Close price position in daily range (0-1)
+------+----------+------------------+------------------+------------------+------------------+-------+---------+--------------------+
|ticker|date      |open              |high              |low               |close             |is_doji|is_hammer|price_position      |
+------+----------+------------------+------------------+------------------+------------------+-------+---------+--------------------+
|AMD   |2024-11-13|142.86000061035156|144.49000549316406|139.07000732421875|139.3000030517578 |0      |0        |0.042434650413141706|
|AMD   |2024-11-14|140.33999633789062|141.39999389648438|138.55999755859375|138.83999633789062|0      |0        |0.09859124660169136 |
|AMD   |2024-11-15|136.570007

In [19]:
# ======================================================================
# CELL 15: Support & Resistance Levels
# ======================================================================
print("=" * 70)
print("CREATING SUPPORT & RESISTANCE LEVELS")
print("=" * 70)

# Support: Rolling minimum of lows over a period
# Resistance: Rolling maximum of highs over a period

for period in [20, 50]:
    window_sr = Window.partitionBy("ticker").orderBy("date").rowsBetween(-(period-1), 0)
    
    # Support levels (rolling minimum of lows)
    df = df.withColumn(f"support_{period}d", spark_min("low").over(window_sr))
    
    # Resistance levels (rolling maximum of highs)
    df = df.withColumn(f"resistance_{period}d", spark_max("high").over(window_sr))
    
    # Distance to support and resistance
    df = df.withColumn(
        f"distance_to_support_{period}d",
        ((col("close") - col(f"support_{period}d")) / col("close")) * 100
    )
    df = df.withColumn(
        f"distance_to_resistance_{period}d",
        ((col(f"resistance_{period}d") - col("close")) / col("close")) * 100
    )

print("✅ Created support & resistance levels:")
print("  - support_20d, support_50d: Rolling minimum lows")
print("  - resistance_20d, resistance_50d: Rolling maximum highs")
print("  - distance_to_support_*: Percentage distance to support")
print("  - distance_to_resistance_*: Percentage distance to resistance")

df.select("ticker", "date", "close", "support_20d", "resistance_20d", 
          "distance_to_support_20d", "distance_to_resistance_20d").show(10, truncate=False)


CREATING SUPPORT & RESISTANCE LEVELS
✅ Created support & resistance levels:
  - support_20d, support_50d: Rolling minimum lows
  - resistance_20d, resistance_50d: Rolling maximum highs
  - distance_to_support_*: Percentage distance to support
  - distance_to_resistance_*: Percentage distance to resistance
+------+----------+------------------+------------------+------------------+-----------------------+--------------------------+
|ticker|date      |close             |support_20d       |resistance_20d    |distance_to_support_20d|distance_to_resistance_20d|
+------+----------+------------------+------------------+------------------+-----------------------+--------------------------+
|AAPL  |2024-11-13|225.1199951171875 |222.75999450683594|226.64999389648438|1.0483300735339172     |0.6796369991481322        |
|AAPL  |2024-11-14|228.22000122070312|222.75999450683594|228.8699951171875 |2.3924312876446865     |0.28481022391012517       |
|AAPL  |2024-11-15|225.0             |222.75999450683

In [20]:
# ======================================================================
# CELL 16: Final Validation - Check for NULLs in Key Features
# ======================================================================
print("=" * 70)
print("FINAL VALIDATION - KEY FEATURES NULL CHECK")
print("=" * 70)

# Key features to check
key_features = [
    "ticker", "date", "open", "high", "low", "close", "volume",
    "prev_close", "price_change", "price_change_pct",
    "sma_20", "ema_20", "rsi_14", "macd", "bb_middle"
]

print("\nNULL counts in key features:")
null_check = df.select([count(when(col(c).isNull(), c)).alias(c) for c in key_features if c in df.columns])
null_check.show(vertical=True)

# Check for any rows with NULLs in critical fields
critical_fields = ["ticker", "date", "close"]
null_critical = df.filter(
    reduce(lambda a, b: a | b, [col(f).isNull() for f in critical_fields if f in df.columns])
)
null_critical_count = null_critical.count()

if null_critical_count > 0:
    print(f"\n⚠️  WARNING: {null_critical_count} rows have NULLs in critical fields (ticker, date, close)")
    null_critical.show(10)
else:
    print(f"\n✅ No NULLs in critical fields (ticker, date, close)")

print(f"\nTotal records: {df.count():,}")
print(f"Unique tickers: {df.select('ticker').distinct().count()}")


FINAL VALIDATION - KEY FEATURES NULL CHECK

NULL counts in key features:


-RECORD 0---------------
 ticker           | 0   
 date             | 0   
 open             | 0   
 high             | 0   
 low              | 0   
 close            | 0   
 volume           | 0   
 prev_close       | 15  
 price_change     | 15  
 price_change_pct | 15  
 sma_20           | 0   
 ema_20           | 0   
 rsi_14           | 22  
 macd             | 0   
 bb_middle        | 0   


✅ No NULLs in critical fields (ticker, date, close)

Total records: 3,750
Unique tickers: 15


In [23]:
# ======================================================================
# CELL 17: Data Quality Check - Duplicates and Completeness
# ======================================================================
print("=" * 70)
print("DATA QUALITY CHECK - DUPLICATES AND COMPLETENESS")
print("=" * 70)

# Check for duplicates
duplicate_count = df.groupBy("ticker", "date").count().filter(col("count") > 1).count()
print(f"\n1. Duplicate Check:")
print(f"   Duplicate (ticker, date) combinations: {duplicate_count}")

if duplicate_count > 0:
    print("   ⚠️  WARNING: Duplicates found!")
    duplicates = df.groupBy("ticker", "date").count().filter(col("count") > 1)
    duplicates.show(20)
    # Remove duplicates, keeping first occurrence
    window_dedup = Window.partitionBy("ticker", "date").orderBy("date")
    df = df.withColumn("row_num", row_number().over(window_dedup))
    df = df.filter(col("row_num") == 1).drop("row_num")
    print(f"   ✅ Removed duplicates. New record count: {df.count():,}")
else:
    print("   ✅ No duplicates found")

# Completeness check
print(f"\n2. Completeness Check:")
completeness_final = df.groupBy("ticker").agg(
    count("date").alias("record_count"),
    spark_min("date").alias("min_date"),
    spark_max("date").alias("max_date"),
    countDistinct("date").alias("unique_dates")
).orderBy("ticker")

completeness_final.show(truncate=False)

# Check for data quality issues
print(f"\n3. Data Quality Issues:")
# Check for negative prices
negative_prices = df.filter((col("close") < 0) | (col("open") < 0) | (col("high") < 0) | (col("low") < 0)).count()
print(f"   Rows with negative prices: {negative_prices}")

# Check for high > low violations
invalid_ohlc_df = df.filter((col("high") < col("low")) | (col("high") < col("open")) | (col("high") < col("close")) | 
                             (col("low") > col("open")) | (col("low") > col("close")))
invalid_ohlc = invalid_ohlc_df.count()
print(f"   Rows with invalid OHLC relationships: {invalid_ohlc}")

# Show the problematic row(s) if any
if invalid_ohlc > 0:
    print("\n   ⚠️  Invalid OHLC row(s) found:")
    invalid_ohlc_df.select("ticker", "date", "open", "high", "low", "close", "volume").show(truncate=False)
    print("\n   Checking which condition(s) are violated:")
    invalid_ohlc_df.select(
        "ticker", "date",
        when(col("high") < col("low"), lit("high < low")).otherwise(lit("")),
        when(col("high") < col("open"), lit("high < open")).otherwise(lit("")),
        when(col("high") < col("close"), lit("high < close")).otherwise(lit("")),
        when(col("low") > col("open"), lit("low > open")).otherwise(lit("")),
        when(col("low") > col("close"), lit("low > close")).otherwise(lit(""))
    ).show(truncate=False)

# Check for zero volume
zero_volume = df.filter(col("volume") == 0).count()
print(f"   Rows with zero volume: {zero_volume}")

if negative_prices == 0 and invalid_ohlc == 0:
    print("   ✅ No data quality issues found")
else:
    print("   ⚠️  Data quality issues detected - fixing OHLC relationships...")
    
    # Fix OHLC relationships to ensure data integrity
    # High should be >= max(open, close, low)
    # Low should be <= min(open, close, high)
    from pyspark.sql.functions import greatest, least
    
    # First, fix high to be at least as high as open, close, and low
    df = df.withColumn("high", greatest(col("high"), col("open"), col("close"), col("low")))
    
    # Then, fix low to be at most as low as open, close, and the updated high
    df = df.withColumn("low", least(col("low"), col("open"), col("close"), col("high")))
    
    # Verify the fix
    invalid_ohlc_after = df.filter((col("high") < col("low")) | (col("high") < col("open")) | (col("high") < col("close")) | 
                                    (col("low") > col("open")) | (col("low") > col("close"))).count()
    
    if invalid_ohlc_after == 0:
        print(f"   ✅ Fixed {invalid_ohlc} invalid OHLC relationship(s)")
    else:
        print(f"   ⚠️  Warning: {invalid_ohlc_after} invalid OHLC relationship(s) remain after fix")


DATA QUALITY CHECK - DUPLICATES AND COMPLETENESS

1. Duplicate Check:
   Duplicate (ticker, date) combinations: 0
   ✅ No duplicates found

2. Completeness Check:
+------+------------+----------+----------+------------+
|ticker|record_count|min_date  |max_date  |unique_dates|
+------+------------+----------+----------+------------+
|AAPL  |250         |2024-11-13|2025-11-12|250         |
|AMD   |250         |2024-11-13|2025-11-12|250         |
|AMZN  |250         |2024-11-13|2025-11-12|250         |
|GOOGL |250         |2024-11-13|2025-11-12|250         |
|INTC  |250         |2024-11-13|2025-11-12|250         |
|JPM   |250         |2024-11-13|2025-11-12|250         |
|META  |250         |2024-11-13|2025-11-12|250         |
|MSFT  |250         |2024-11-13|2025-11-12|250         |
|NVDA  |250         |2024-11-13|2025-11-12|250         |
|ORCL  |250         |2024-11-13|2025-11-12|250         |
|QQQ   |250         |2024-11-13|2025-11-12|250         |
|SPY   |250         |2024-11-13|2025-11

In [24]:
# ======================================================================
# CELL 18: Additional Data Checks and Feature Engineering
# ======================================================================
print("=" * 70)
print("ADDITIONAL DATA CHECKS AND FEATURE ENGINEERING")
print("=" * 70)

# Add date_key for joining with sentiment data
df = df.withColumn("date_key", date_format(col("date"), "yyyy-MM-dd"))

# Add ingestion timestamp
from pyspark.sql.functions import current_timestamp
df = df.withColumn("ingestion_timestamp", current_timestamp())

# Round numeric columns to reasonable precision
numeric_cols = [f.name for f in df.schema.fields if isinstance(f.dataType, (DoubleType, FloatType, DecimalType))]
for col_name in numeric_cols:
    if col_name not in ["ticker", "date", "date_key", "ingestion_timestamp"]:
        df = df.withColumn(col_name, spark_round(col(col_name), 4))

# Reorder first 8 columns, keep rest in current order
priority_columns = ["ticker", "date", "date_key", "open", "high", "low", "close", "volume"]
all_columns = df.columns

# Get priority columns that exist
priority_existing = [c for c in priority_columns if c in all_columns]

# Get remaining columns (not in priority list) in their current order
remaining_columns = [c for c in all_columns if c not in priority_columns]

# Combine: priority columns first, then remaining columns
final_column_order = priority_existing + remaining_columns

# Reorder DataFrame
df = df.select(*final_column_order)

print("✅ Final feature engineering complete")
print(f"\nFinal schema ({len(df.columns)} columns):")
df.printSchema()

print(f"\nFinal record count: {df.count():,}")
print(f"Unique tickers: {df.select('ticker').distinct().count()}")


ADDITIONAL DATA CHECKS AND FEATURE ENGINEERING
✅ Final feature engineering complete

Final schema (68 columns):
root
 |-- ticker: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_key: string (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: long (nullable = true)
 |-- prev_close: double (nullable = true)
 |-- price_change: double (nullable = true)
 |-- price_change_pct: double (nullable = true)
 |-- sma_5: double (nullable = true)
 |-- sma_10: double (nullable = true)
 |-- sma_20: double (nullable = true)
 |-- sma_50: double (nullable = true)
 |-- ema_5: float (nullable = true)
 |-- ema_10: float (nullable = true)
 |-- ema_20: float (nullable = true)
 |-- ema_50: float (nullable = true)
 |-- daily_range: double (nullable = true)
 |-- daily_range_pct: double (nullable = true)
 |-- volatility_5d: double (nullable = true)
 |-- volatility_10d: dou

In [25]:
# ======================================================================
# CELL 19: Partition by Stock, Sort by Date, and Write to BigQuery
# ======================================================================
print("=" * 70)
print("PREPARING DATA FOR BIGQUERY")
print("=" * 70)

# Partition by ticker and sort by date (oldest to newest)
window_partition = Window.partitionBy("ticker").orderBy("date")

# Add row number to ensure proper ordering
df = df.withColumn("_row_num", row_number().over(window_partition))

# Sort the entire DataFrame by ticker and date
df_final = df.orderBy("ticker", "date").drop("_row_num")

print("✅ Data partitioned by ticker and sorted by date (oldest to newest)")
print(f"\nFinal record count: {df_final.count():,}")

# Show sample to verify sorting
print("\nSample data (first 10 rows):")
df_final.show(10, truncate=False)

# Verify sorting
print("\nVerifying sort order (sample by ticker):")
for ticker in df_final.select("ticker").distinct().limit(3).rdd.map(lambda r: r[0]).collect():
    sample = df_final.filter(col("ticker") == ticker).select("ticker", "date", "close").limit(5)
    print(f"\n{ticker}:")
    sample.show(truncate=False)


PREPARING DATA FOR BIGQUERY
✅ Data partitioned by ticker and sorted by date (oldest to newest)

Final record count: 3,750

Sample data (first 10 rows):


25/11/14 07:33:51 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------+----------+----------+------+------+------+------+--------+----------+------------+----------------+--------+--------+--------+--------+--------+--------+--------+--------+-----------+---------------+-------------+--------------+--------------+---------------+---------------+---------------+------------+---------------+--------+-------+-----------+---------------+------------+----------------+---------+------+--------+--------+--------+------------+--------+--------+--------+------+-----------+--------------+----------+-----------------+----------------+----------------+-----------+--------+------------+-------+---------+--------------------+--------------------+--------------+-----------+--------------+-----------------------+--------------------------+-----------+--------------+-----------------------+--------------------------+--------------------------+
|ticker|date      |date_key  |open  |high  |low   |close |volume  |prev_close|price_change|price_change_pct|sma_5   |sma_1


GOOGL:
+------+----------+------+
|ticker|date      |close |
+------+----------+------+
|GOOGL |2024-11-13|178.88|
|GOOGL |2024-11-14|175.58|
|GOOGL |2024-11-15|172.49|
|GOOGL |2024-11-18|175.3 |
|GOOGL |2024-11-19|178.12|
+------+----------+------+


AMD:
+------+----------+------+
|ticker|date      |close |
+------+----------+------+
|AMD   |2024-11-13|139.3 |
|AMD   |2024-11-14|138.84|
|AMD   |2024-11-15|134.9 |
|AMD   |2024-11-18|138.93|
|AMD   |2024-11-19|139.39|
+------+----------+------+


TSLA:
+------+----------+------+
|ticker|date      |close |
+------+----------+------+
|TSLA  |2024-11-13|330.24|
|TSLA  |2024-11-14|311.18|
|TSLA  |2024-11-15|320.72|
|TSLA  |2024-11-18|338.74|
|TSLA  |2024-11-19|346.0 |
+------+----------+------+



In [27]:
# ======================================================================
# CELL 20: Write to BigQuery
# ======================================================================
print("=" * 70)
print("WRITING TO BIGQUERY")
print("=" * 70)

# Get BigQuery table schema to ensure column compatibility
table_id = f"{BIGQUERY_PROJECT}.{BIGQUERY_DATASET}.{BIGQUERY_TABLE_STOCKS}"
table = bq_client.get_table(table_id)
bq_schema_columns = [field.name for field in table.schema]

print(f"BigQuery table has {len(bq_schema_columns)} columns")

# Convert Spark DataFrame to Pandas for BigQuery write
print("Converting Spark DataFrame to Pandas...")
df_pandas = df_final.toPandas()

print(f"Pandas DataFrame shape before filtering: {df_pandas.shape}")

# Filter to only include columns that exist in BigQuery schema
columns_to_keep = [col for col in df_pandas.columns if col in bq_schema_columns]
columns_to_drop = [col for col in df_pandas.columns if col not in bq_schema_columns]

if columns_to_drop:
    print(f"\n⚠️  Dropping {len(columns_to_drop)} column(s) not in BigQuery schema: {columns_to_drop}")
    df_pandas = df_pandas[columns_to_keep]

print(f"Pandas DataFrame shape after filtering: {df_pandas.shape}")

# Convert date columns to proper format for BigQuery
if 'date' in df_pandas.columns:
    df_pandas['date'] = pd.to_datetime(df_pandas['date']).dt.date
if 'date_key' in df_pandas.columns:
    df_pandas['date_key'] = pd.to_datetime(df_pandas['date_key']).dt.date

# Convert timestamp columns
if 'ingestion_timestamp' in df_pandas.columns:
    df_pandas['ingestion_timestamp'] = pd.to_datetime(df_pandas['ingestion_timestamp'])

print(f"\nWriting {len(df_pandas):,} records to BigQuery...")
print(f"Table: {table_id}")

job = bq_client.load_table_from_dataframe(
    df_pandas,
    table_id,
    job_config=bigquery.LoadJobConfig(
        write_disposition=bigquery.WriteDisposition.WRITE_APPEND,
        create_disposition=bigquery.CreateDisposition.CREATE_NEVER
    )
)
job.result()  # Wait for the job to complete

print(f"✅ Successfully wrote {len(df_pandas):,} records to BigQuery")
print(f"Table: {table_id}")

# Verify write
table = bq_client.get_table(table_id)
print(f"\n✅ Verification:")
print(f"   Table rows: {table.num_rows:,}")
print(f"   Table size: {table.num_bytes / (1024*1024):.2f} MB")


WRITING TO BIGQUERY
BigQuery table has 63 columns
Converting Spark DataFrame to Pandas...
Pandas DataFrame shape before filtering: (3750, 68)

⚠️  Dropping 5 column(s) not in BigQuery schema: ['bb_std', 'ema_12', 'ema_26', 'ema_9', 'running_max']
Pandas DataFrame shape after filtering: (3750, 63)

Writing 3,750 records to BigQuery...
Table: solid-coral-469323-i0.stock_sentiment.stock_prices_silver


/Users/evancallaghan/data_portfolio/data_engineering/stock_x_sentiment/venv311/lib/python3.11/site-packages/google/cloud/bigquery/_pandas_helpers.py:484: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


✅ Successfully wrote 3,750 records to BigQuery
Table: solid-coral-469323-i0.stock_sentiment.stock_prices_silver

✅ Verification:
   Table rows: 3,750
   Table size: 1.79 MB


In [ ]:
# ======================================================================
# CELL 21: Validation and Summary
# ======================================================================
print("=" * 70)
print("PIPELINE SUMMARY")
print("=" * 70)

# Query BigQuery to verify
query = f"""
SELECT 
    COUNT(*) as total_records,
    COUNT(DISTINCT ticker) as unique_tickers,
    COUNT(DISTINCT date) as unique_dates,
    MIN(date) as min_date,
    MAX(date) as max_date,
    AVG(close) as avg_close_price,
    AVG(volume) as avg_volume
FROM `{BIGQUERY_PROJECT}.{BIGQUERY_DATASET}.{BIGQUERY_TABLE_STOCKS}`
"""

results = bq_client.query(query).to_dataframe()
print(results.to_string())

print("\n✅ Pipeline completed successfully!")
print("=" * 70)


PIPELINE SUMMARY
   total_records  unique_tickers  unique_dates    min_date    max_date  avg_close_price    avg_volume
0           3750              15           250  2024-11-13  2025-11-12       306.533109  5.320986e+07

✅ Pipeline completed successfully!


/Users/evancallaghan/data_portfolio/data_engineering/stock_x_sentiment/venv311/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


25/11/14 12:52:13 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 147376 ms exceeds timeout 120000 ms
25/11/14 12:52:13 WARN SparkContext: Killing executors is not supported by current scheduler.
25/11/14 12:52:16 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$